In [ ]:
%reset -f

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import os

from datetime import datetime

In [ ]:
filename = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Input/Current_Kumho_FUEL.csv"
x1 = pd.read_csv(filename)
Kumho_All_Data = pd.DataFrame(x1)
Kumho_Q1_Data = Kumho_All_Data[Kumho_All_Data['CalendarYearQuarter'].str.contains('Q1')].reset_index().drop('index', axis =1)
Kumho_Q2_Data = Kumho_All_Data[Kumho_All_Data['CalendarYearQuarter'].str.contains('Q2')].reset_index().drop('index', axis =1)
Kumho_Q3_Data = Kumho_All_Data[Kumho_All_Data['CalendarYearQuarter'].str.contains('Q3')].reset_index().drop('index', axis =1)
Kumho_Q4_Data = Kumho_All_Data[Kumho_All_Data['CalendarYearQuarter'].str.contains('Q4')].reset_index().drop('index', axis =1)

In [ ]:
custKey = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Updated Combined CMP Key.xlsx"
x2 = pd.read_excel(custKey)
All_Cust_List = pd.DataFrame(x2)
Kumho_Cust_List = All_Cust_List[All_Cust_List['Program Company'].str.contains('Kumho')].drop(
    ['CustID','CustAddID','CustID-CustAddID','Program Company','Program Dealer Name','Programs Title','Active Sales?',
     'Report Filter', 'Goodyear TSN'], axis=1).reset_index([]).rename(index=str, columns={
        'CustAddKey':'CustomerAddressKey'})

In [ ]:
tt = datetime.now().timetuple()

In [ ]:
currentKumhoData =pd.DataFrame([])
Custlist =pd.DataFrame([])
if (tt.tm_mon == 1) | (tt.tm_mon == 2) | (tt.tm_mon == 3):
    if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
        quarter = 1
        daysINqtr = 91
        currentKumhoData = Kumho_Q1_Data
        print('Leap year')
    else:
        quarter = 1
        daysINqtr = 90       
        currentKumhoData = Kumho_Q1_Data
elif (tt.tm_mon == 4) | (tt.tm_mon == 5) | (tt.tm_mon == 6):
    quarter = 2
    daysINqtr = 91
    subtractDays = 90          
    currentKumhoData = Kumho_Q2_Data
elif (tt.tm_mon == 7) | (tt.tm_mon == 8) | (tt.tm_mon == 9):
    quarter = 3
    daysINqtr = 92
    subtractDays = 90 + 91
    currentKumhoData = Kumho_Q3_Data
else:
    quarter = 4
    daysINqtr = 92
    subtractDays = 90 + 91 + 92
    currentKumhoData = Kumho_Q4_Data

In [ ]:
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/KUMHO_FUEL_Incentive.txt'
df_FUEL_tierMap = pd.read_csv(filename2, sep = '\t')

filename3 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/KUMHO_VOL_Incentive.txt'
df_VOL_tierMap = pd.read_csv(filename3, sep = '\t')

In [ ]:
tup_now = datetime.now().timetuple()
currentYear = datetime.now().year

In [ ]:
ts_currentData = os.path.getmtime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
# print(dt_currentData)
ts_FUEL_incentive = os.path.getctime(filename2)
dt_FUEL_incentive = datetime.fromtimestamp(ts_FUEL_incentive)
# print(dt_FUEL_incentive)
ts_VOL_incentive = os.path.getctime(filename3)
dt_VOL_incentive = datetime.fromtimestamp(ts_VOL_incentive)
# print(dt_VOL_incentive)

In [ ]:
Custlist =pd.DataFrame([])
currentKumhoFUELData = pd.DataFrame([])
CustGroupAll_VOL = currentKumhoData.groupby(
    'CustomerAddressKey').sum()
currentKumhoVolData = currentKumhoData.merge(
    CustGroupAll_VOL, on='CustomerAddressKey').drop(['Net_Qty_x','Rebate_Class','Rim_Class'], axis=1).rename(
    index=str, columns={'Net_Qty_y':'Total_Net_Qty'}).drop_duplicates().reset_index(drop=True)


currentKumhoData_PCR = currentKumhoData[currentKumhoData['Rebate_Class'].str.contains('PCR')].reset_index(
    ).drop('index', axis =1)
CustGroupAll_FUEL_PCR = currentKumhoData_PCR.groupby(
    'CustomerAddressKey').sum().rename(index=str, columns={'Net_Qty':'Net_Qty_PCR'})

currentKumhoData_LTSUV = currentKumhoData[currentKumhoData['Rebate_Class'].str.contains('LT/SUV')].reset_index(
    ).drop('index', axis =1)
CustGroupAll_FUEL_LTSUV = currentKumhoData_LTSUV.groupby(
    'CustomerAddressKey').sum().rename(index=str, columns={'Net_Qty':'Net_Qty_LTSUV'})

currentKumhoData_UHP = currentKumhoData[currentKumhoData['Rebate_Class'].str.contains('UHP')].reset_index(
    ).drop('index', axis =1)
CustGroupAll_FUEL_UHP = currentKumhoData_UHP.groupby(
    'CustomerAddressKey').sum().rename(index=str, columns={'Net_Qty':'Net_Qty_UHP'})

currentKumhoData_18plus = currentKumhoData[currentKumhoData['Rim_Class'].str.contains('18plus')].reset_index(
    ).drop('index', axis=1)
CustGroupAll_FUEL_18plus = currentKumhoData_18plus.groupby(
    'CustomerAddressKey').sum().rename(index=str, columns={'Net_Qty':'Net_Qty_18plus'})

Custlist = currentKumhoData.drop(['Net_Qty', 'Rebate_Class', 'Rim_Class','CalendarYearQuarter'], axis=1).drop_duplicates()
Custlist.set_index('CustomerAddressKey')

dfs = [CustGroupAll_FUEL_PCR, CustGroupAll_FUEL_LTSUV, CustGroupAll_FUEL_UHP, CustGroupAll_FUEL_18plus]

currentKumhoFUELData = pd.concat(dfs, axis=1,sort=True)
currentKumhoFUELData = currentKumhoFUELData.fillna(0).reset_index().rename(columns={'index':'CustomerAddressKey'})
currentKumhoFUELData['CustomerAddressKey'] = pd.to_numeric(currentKumhoFUELData['CustomerAddressKey'])
currentKumhoFUELData = currentKumhoFUELData.sort_values(by=['CustomerAddressKey'], ascending=True).reset_index(drop=True)

In [ ]:
calculatedDataFUEL = pd.DataFrame([])
for ii in range(currentKumhoFUELData.shape[0]):
# for ii in range(792,793):    

    singleCustyFUEL = currentKumhoFUELData.iloc[[ii]].assign(QTDsum=0, nextTierFUEL=0, TierFUEL=0, current_total_rebate=0)

    singleCustyFUEL.reset_index(inplace=True,drop=True)
    
    singleCustyFUEL.QTDsum_PCR = currentKumhoFUELData.iloc[ii,1]
    singleCustyFUEL.QTDsum_LTSUV = currentKumhoFUELData.iloc[ii,2]
    singleCustyFUEL.QTDsum_UHP = currentKumhoFUELData.iloc[ii,3]
    singleCustyFUEL.QTDsum_18plus = currentKumhoFUELData.iloc[ii,4]
    singleCustyFUEL.QTDsum_TOTAL = singleCustyFUEL.QTDsum_PCR + singleCustyFUEL.QTDsum_LTSUV + singleCustyFUEL.QTDsum_UHP

    if  tt.tm_year == currentYear:
        numDays = tt.tm_yday - subtractDays # get number of days
        estimatedTires_PCR =  np.floor((singleCustyFUEL.QTDsum_PCR/(numDays/daysINqtr)))
        estimatedTires_LTSUV = np.floor((singleCustyFUEL.QTDsum_LTSUV/(numDays/daysINqtr)))
        estimatedTires_UHP = np.floor((singleCustyFUEL.QTDsum_UHP/(numDays/daysINqtr)))
        estimatedTires_18plus = np.floor((singleCustyFUEL.QTDsum_18plus/(numDays/daysINqtr)))
        estimatedTires_TOTAL = estimatedTires_PCR + estimatedTires_LTSUV + estimatedTires_UHP
        

        if (singleCustyFUEL.QTDsum_TOTAL >= (df_FUEL_tierMap.iloc[1,5])) & (singleCustyFUEL.QTDsum_TOTAL <= (df_FUEL_tierMap.iloc[1,6])):
            TierFUEL = 1
        elif (singleCustyFUEL.QTDsum_TOTAL >= (df_FUEL_tierMap.iloc[2,5])) & (singleCustyFUEL.QTDsum_TOTAL <= (df_FUEL_tierMap.iloc[2,6])):
            TierFUEL = 2
        elif (singleCustyFUEL.QTDsum_TOTAL >= (df_FUEL_tierMap.iloc[3,5])):
            TierFUEL = 3
        else:
            TierFUEL = 0
        
      
        
        nextTierFUEL = TierFUEL + 1
        unitsTonextTierFUEL = df_FUEL_tierMap.iloc[nextTierFUEL,5] - singleCustyFUEL.QTDsum_TOTAL
         
        if   (estimatedTires_TOTAL >= (df_FUEL_tierMap.iloc[1,5] )) & (estimatedTires_TOTAL <= (df_FUEL_tierMap.iloc[1,6])):
            tierTypeFUEL = 1
        elif (estimatedTires_TOTAL >= (df_FUEL_tierMap.iloc[2,5] )) & (estimatedTires_TOTAL <= (df_FUEL_tierMap.iloc[2,6])):
            tierTypeFUEL = 2
        elif (estimatedTires_TOTAL >= (df_FUEL_tierMap.iloc[3,5] )):
            tierTypeFUEL = 3
        else:
            tierTypeFUEL = 0
        
        if tierTypeFUEL < 3:
            tierUPFUEL = tierTypeFUEL + 1
        else:
            tierUPFUEL = 3
        
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
           
         # floor to underestimate estimated tires
        total_Net_Qty = singleCustyFUEL.QTDsum_TOTAL
        total_est_tires = estimatedTires_TOTAL
        
        current_tires_rebate_PCR = df_FUEL_tierMap.iloc[TierFUEL,3] * singleCustyFUEL.QTDsum_PCR   
        current_tires_rebate_LTSUV = df_FUEL_tierMap.iloc[TierFUEL,2] * singleCustyFUEL.QTDsum_LTSUV
        current_tires_rebate_UHP = df_FUEL_tierMap.iloc[TierFUEL,1] * singleCustyFUEL.QTDsum_UHP
        current_tires_rebate_18plus = df_FUEL_tierMap.iloc[TierFUEL,4] * singleCustyFUEL.QTDsum_18plus
        total_current_tires_rebate = current_tires_rebate_PCR + current_tires_rebate_LTSUV + current_tires_rebate_UHP + current_tires_rebate_18plus
        
        est_tires_rebate_PCR = df_FUEL_tierMap.iloc[tierTypeFUEL,3] * estimatedTires_PCR   
        est_tires_rebate_LTSUV = df_FUEL_tierMap.iloc[tierTypeFUEL,2] * estimatedTires_LTSUV
        est_tires_rebate_UHP = df_FUEL_tierMap.iloc[tierTypeFUEL,1] * estimatedTires_UHP
        est_tires_rebate_18plus = df_FUEL_tierMap.iloc[tierTypeFUEL,4] * estimatedTires_18plus
        total_est_tires_rebate = est_tires_rebate_PCR + est_tires_rebate_LTSUV + est_tires_rebate_UHP + est_tires_rebate_18plus

         # floor to underestimate estimated tires
        PCR_prop = singleCustyFUEL.QTDsum_PCR/singleCustyFUEL.QTDsum_TOTAL
        LTSUV_prop = singleCustyFUEL.QTDsum_LTSUV/singleCustyFUEL.QTDsum_TOTAL
        UHP_prop = singleCustyFUEL.QTDsum_UHP/singleCustyFUEL.QTDsum_TOTAL
        plus18_prop = singleCustyFUEL.QTDsum_18plus/singleCustyFUEL.QTDsum_TOTAL
        
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        PCR_1up_tires = np.ceil((PCR_prop*df_FUEL_tierMap.iloc[tierUPFUEL,5]))
        LTSUV_1up_tires = np.ceil((LTSUV_prop*df_FUEL_tierMap.iloc[tierUPFUEL,5]))
        UHP_1up_tires = np.ceil((UHP_prop*df_FUEL_tierMap.iloc[tierUPFUEL,5]))
        plus18_1up_tires = np.ceil((plus18_prop*df_FUEL_tierMap.iloc[tierUPFUEL,5]))
        total_1up_tires = df_FUEL_tierMap.iloc[tierUPFUEL,5]
        
        
        PCR_1up_tires_rebate = PCR_1up_tires * df_FUEL_tierMap.iloc[tierUPFUEL,3]
        LTSUV_1up_tires_rebate = LTSUV_1up_tires * df_FUEL_tierMap.iloc[tierUPFUEL,2]
        UHP_1up_tires_rebate = UHP_1up_tires * df_FUEL_tierMap.iloc[tierUPFUEL,1]
        plus18_1up_tires_rebate = plus18_1up_tires * df_FUEL_tierMap.iloc[tierUPFUEL,4]
        total_1up_tires_rebate = PCR_1up_tires_rebate + LTSUV_1up_tires_rebate + UHP_1up_tires_rebate + plus18_1up_tires_rebate

    
        if estimatedTires_TOTAL > np.ceil((df_FUEL_tierMap.iloc[tierUPFUEL,5])):
            total_1up_tires = estimatedTires_TOTAL
            total_1up_tires_rebate = total_est_tires_rebate

        # ERROR: if using all scalar values pass an index.  Convert one scalar to a list, e.g. [var]
        tempData = pd.DataFrame(
            {'estimated_tier':[tierTypeFUEL], 'current_est_tires':[total_est_tires], 'total_est_rebate': [total_est_tires_rebate], 
             'Up_tier':[tierUPFUEL], 'total_up_tires':[total_1up_tires], 'total_up_rebate':[total_1up_tires_rebate], 
             'period': [quarter], 'grain': 'Quarter','unitsTonextTier':[unitsTonextTierFUEL], 'current_Tier':[TierFUEL], 
             'next_Tier':[nextTierFUEL], 'Total_Net_Qty':total_Net_Qty, 'current_total_rebate':total_current_tires_rebate})
        calculatedDataFUEL = calculatedDataFUEL.append(tempData)
        
    else: 
        tempData = pd.DataFrame(
            {'estimated_tier':0, 'current_est_tires':0, 'total_est_rebate': 0, 
             'Up_tier':1, 'total_up_tires':df_tierMap.iloc[1,2], 'total_up_rebate':0, 
             'period': quarter, 'grain': 'Quarter','unitsTonextTierFUEL':0, 'current_Tier':0,
             'next_Tier':0, 'Total_Net_Qty':0, 'current_total_rebate':0})
        calculatedDataFUEL = calculatedDataFUEL.append(tempData)   
###########################################################################################################################        
calculatedDataFUEL = calculatedDataFUEL.reset_index(drop=True)
compliedDataFUEL = pd.concat([calculatedDataFUEL, currentKumhoFUELData],axis=1).sort_values(by=['CustomerAddressKey'], ascending=True).reset_index(drop=True)
compliedDataFUEL['Date'] = dt_currentData

In [ ]:
finalDataFUEL = pd.merge(compliedDataFUEL, Kumho_Cust_List, on='CustomerAddressKey', how='inner').drop_duplicates().sort_values(by=['CustomerAddressKey']).reset_index(drop=True)
finalDataFUEL = finalDataFUEL.replace([np.inf, -np.inf], np.nan).fillna(0)
# finalDataFUEL

In [ ]:
pathFUEL = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = pathFUEL + 'CMP_KUMHO_FUEL.csv'
finalDataFUEL.to_csv(writeFilename, index=False)
writeFilename_archive = pathFUEL + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_KUMHO_FUEL_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalDataFUEL.to_csv(writeFilename_archive, index=False)

In [ ]:
calculatedDataVOL = pd.DataFrame([])
for ii in range(currentKumhoVolData.shape[0]):
# for ii in range(2,3):    

    singleCustyVOL = currentKumhoVolData.iloc[[ii]].assign(QTDsum=0, nextTierVOL=0, TierVOL=0)

    singleCustyVOL.reset_index(inplace=True,drop=True)
 
    singleCustyVOL.QTDsum = currentKumhoVolData.iloc[ii,2]
    
#     singleCustyVOL_select = singleCustyVOL[
#         ['CustomerAddressKey','CalendarYearQuarter','Total_Net_Qty','nextTierVOL']]
#     singleCustyVOL_select = singleCustyVOL_select.rename(index=str, columns={
#         'CustomerAddressKey':'CustomerAddressKey','CalendarYearQuarter':'QTR','Total_Net_Qty':'Total_Qty', 'nextTierVOL':'nextTierVOL'})

    if  tt.tm_year == currentYear:
        numDays = tt.tm_yday - subtractDays # get number of days
        estimatedTires = np.floor((singleCustyVOL.QTDsum/(numDays/daysINqtr))).values
        if (singleCustyVOL.QTDsum.values >= (df_VOL_tierMap.iloc[1,2])) & (singleCustyVOL.QTDsum.values <= (df_VOL_tierMap.iloc[1,3])):
            TierVOL = 1
        elif (singleCustyVOL.QTDsum.values >= (df_VOL_tierMap.iloc[2,2])) & (singleCustyVOL.QTDsum.values <= (df_VOL_tierMap.iloc[2,3])):
            TierVOL = 2
        elif (singleCustyVOL.QTDsum.values >= (df_VOL_tierMap.iloc[3,2])) & (singleCustyVOL.QTDsum.values <= (df_VOL_tierMap.iloc[3,3])):
            TierVOL = 3
        elif (singleCustyVOL.QTDsum.values >= (df_VOL_tierMap.iloc[4,2])):
            TierVOL = 4
        else:
            TierVOL = 0
            
        nextTierVOL = TierVOL + 1
        unitsTonextTierVOL = (df_VOL_tierMap.iloc[nextTierVOL,2]) - singleCustyVOL.Total_Net_Qty.values
        
        if   (estimatedTires >= (df_VOL_tierMap.iloc[1,2] )) & (estimatedTires <= (df_VOL_tierMap.iloc[1,3])):
            tierTypeVOL = 1
        elif (estimatedTires >= (df_VOL_tierMap.iloc[2,2] )) & (estimatedTires <= (df_VOL_tierMap.iloc[2,3])):
            tierTypeVOL = 2
        elif (estimatedTires >= (df_VOL_tierMap.iloc[3,2] )) & (estimatedTires <= (df_VOL_tierMap.iloc[3,3])):
            tierTypeVOL = 3
        elif (estimatedTires >= (df_VOL_tierMap.iloc[4,2])):
            tierTypeVOL = 4
        else:
            tierTypeVOL = 0
        
        if tierTypeVOL < 4:
            tierUPVOL = tierTypeVOL + 1
        else:
            tierUPVOL = 4    
            
         # floor to underestimate estimated tires
        total_est_tires = estimatedTires
        
        total_current_tires_rebate = df_VOL_tierMap.iloc[TierVOL,1] * singleCustyVOL.QTDsum
        total_est_tires_rebate = df_VOL_tierMap.iloc[tierTypeVOL,1] * total_est_tires
        
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        total_1up_tires = np.ceil((df_VOL_tierMap.iloc[tierUPVOL,2]))
        total_1up_tires_rebate = np.ceil((df_VOL_tierMap.iloc[tierUPVOL,2])) * df_VOL_tierMap.iloc[tierUPVOL,1]
        
        if estimatedTires > np.ceil((df_VOL_tierMap.iloc[tierUPVOL,2])):
            total_1up_tires = estimatedTires
            total_1up_tires_rebate = total_est_tires_rebate
            
        tempData = pd.DataFrame(
            {'estimated_tier':tierTypeVOL, 'current_est_tires':total_est_tires, 'total_est_rebate': total_est_tires_rebate, 
             'Up_tier':tierUPVOL, 'total_up_tires':total_1up_tires, 'total_up_rebate':total_1up_tires_rebate, 
             'period': quarter, 'grain': 'Quarter','unitsTonextTierVOL':unitsTonextTierVOL, 'current_Tier':TierVOL,
             'next_Tier':nextTierVOL, 'current_total_rebate':total_current_tires_rebate})
        calculatedDataVOL = calculatedDataVOL.append(tempData)
        
    else: 
        tempData = pd.DataFrame(
            {'estimated_tier':0, 'current_est_tires':0, 'total_est_rebate': 0, 
             'Up_tier':1, 'total_up_tires':df_tierMap.iloc[1,2], 'total_up_rebate':0, 
             'period': quarter, 'grain': 'Quarter','unitsTonextTierVOL':0, 'current_Tier':0, 'next_Tier':0, 'current_total_rebate':0})
        calculatedDataVOL = calculatedDataVOL.append(tempData)   
###########################################################################################################################        
calculatedDataVOL = calculatedDataVOL.reset_index(drop=True)
compliedDataVOL = pd.concat([calculatedDataVOL,currentKumhoVolData], axis=1).sort_values(by=['CustomerAddressKey'], ascending=True).reset_index(drop=True)
compliedDataVOL['Date'] = dt_currentData

In [ ]:
finalDataVOL = pd.merge(compliedDataVOL, Kumho_Cust_List, on='CustomerAddressKey', how='inner').drop_duplicates().sort_values(by=['CustomerAddressKey']).reset_index(drop=True)
compliedDataVOL['Date'] = dt_currentData
finalDataVOL = finalDataVOL.replace([np.inf, -np.inf], np.nan).fillna(0)
# finalDataVOL

In [ ]:
pathVOL = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = pathVOL + 'CMP_KUMHO_VOL.csv'
finalDataVOL.to_csv(writeFilename, index=False)
writeFilename_archive = pathVOL + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_KUMHO_VOL_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalDataVOL.to_csv(writeFilename_archive, index=False)